allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)
    Returns True if two arrays are element-wise equal within a tolerance.
   >The tolerance values are positive, typically very small numbers.  The relative difference (`rtol` * abs(`b`)) and the absolute difference
    `atol` are added together to compare against the absolute difference between `a` and `b`.
    >If either array contains one or more NaNs, False is returned.Infs are treated as equal if they are in the same place and of the same
    sign in both arrays.

paddle.optimizer中需要scheduler和optimizer，scheduler.step就可以更新学习率，可以一批或一epoch更新学习率， 自己决定
fluid.optimizer中的optimizer需要optimizer.minimize(loss)，有这个学习率才变化，不然不变


In [1]:
import numpy as np
import paddle
import paddle.fluid as fluid

In [31]:

#不指定学习率退化方法的时候，学习率都一样，哪adam有啥用
# example1: LearningRateDecay is not used, return value is all the same
with fluid.dygraph.guard():
    emb = fluid.dygraph.Embedding([10, 10])
    adam = fluid.optimizer.Adam(0.001, parameter_list = emb.parameters())
    for i in range(10):
        lr = adam.current_step_lr()
        print(lr,end=' ') # 0.001


0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 

In [30]:
from paddle.optimizer.lr import PiecewiseDecay

# example2: PiecewiseDecay is used, return the step learning rate
#with fluid.dygraph.guard():
inp = np.random.uniform(-0.1, 0.1, [10, 10]).astype("float32")
linear = paddle.nn.Linear(10, 10)
inp = paddle.to_tensor(inp)
out = linear(inp)
loss = paddle.mean(out)

bd = [2, 4, 6, 8]
value = [0.2, 0.4, 0.6, 0.8, 1.0]
# epoch:上一轮的轮数，重启训练时设置为上一轮的epoch数。默认值为 -1，则为初始学习率 。
scheduler=PiecewiseDecay(bd,value,last_epoch=-1)
adam = paddle.optimizer.Adam(scheduler,parameters=linear.parameters())

# first step: learning rate is 0.2
np.allclose(adam.get_lr(), 0.2, rtol=1e-06, atol=0.0) # True

# learning rate for different steps
ret = [0.2, 0.2, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0, 1.0, 1.0]
for i in range(12):
    out=linear(inp)
    loss=paddle.mean(out)
    lr = adam.get_lr()
    loss.backward()
    adam.step()
    scheduler.step() #没有这个学习率不会变化
    adam.clear_grad()

    print(f'step:{i} lr:{lr}')
    np.allclose(lr, ret[i], rtol=1e-06, atol=0.0) # True

step:0 lr:0.2
step:1 lr:0.2
step:2 lr:0.4
step:3 lr:0.4
step:4 lr:0.6
step:5 lr:0.6
step:6 lr:0.8
step:7 lr:0.8
step:8 lr:1.0
step:9 lr:1.0
step:10 lr:1.0
step:11 lr:1.0


In [28]:
from paddle.fluid.dygraph import ExponentialDecay

with fluid.dygraph.guard():
    inp = np.random.uniform(-0.1, 0.1, [10, 10]).astype("float32")
    linear = paddle.nn.Linear(10, 10)
    inp = paddle.to_tensor(inp)
    out = linear(inp)
    loss = fluid.layers.reduce_mean(out)
    
    adam = fluid.optimizer.Adam(learning_rate=ExponentialDecay(
              learning_rate=0.01,
              decay_steps=2,
              decay_rate=0.5,
              staircase=True), parameter_list = linear.parameters())
    for i in range(12):
        adam.minimize(loss)#有这个学习率才变化，不然不变
        lr = adam.current_step_lr()

        print('step:{},lr:{:.6f}'.format(i,lr))

step:0,lr:0.010000
step:1,lr:0.010000
step:2,lr:0.005000
step:3,lr:0.005000
step:4,lr:0.002500
step:5,lr:0.002500
step:6,lr:0.001250
step:7,lr:0.001250
step:8,lr:0.000625
step:9,lr:0.000625
step:10,lr:0.000312
step:11,lr:0.000312
